In [36]:
from pathlib import Path
import librosa
import torch
from argparse import ArgumentParser
import matplotlib
import h5py
import tqdm
from IPython.display import Audio, display

import sys
sys.path += ['../src']

import utils
import wavenet_models
from utils import save_audio
from wavenet import WaveNet
from wavenet_generator import WavenetGenerator
from nv_wavenet_generator import NVWavenetGenerator

In [37]:
# Inference params
checkpoint = Path('../checkpoints/pretrained_musicnet/lastmodel')
decoders = [0, 1, 2, 3, 4, 5]
batch_size = 1
rate = 16000
split_size = 20
file_paths = [Path('test.wav')]
print(file_paths)
for file in file_paths:
    print(file)

[WindowsPath('test.wav')]
test.wav


In [38]:
def disp(x, decoder_ix):
    wav = utils.inv_mu_law(x.cpu().numpy())
    print(f'Decoder: {decoder_ix}')
    print(f'X min: {x.min()}, max: {x.max()}')

    display(Audio(wav.squeeze(), rate=rate))
        
def extract_id(path):
    decoder_id = str(path)[:-4].split('_')[-1]
    return int(decoder_id)

In [39]:
class args:
    def __init__(self):
        self.latent_d = 64    # 名称
        self.repeat_num = 6     # 尺寸
        self.encoder_channels = 128     # 列表
        self.encoder_blocks= 3
        self.encoder_pool =  800,
        self.encoder_final_kernel_size =  1
        self.encoder_layers =  10
        self.encoder_func = 'relu' 
        self.blocks= 4
        self.layers = 14  
        self.kernel_size =  2 
        self.skip_channels =  128 
        self.residual_channels =  128  
        self.classes =  256
model_args =args() # 定义结构对象

In [40]:
import torch.nn as nn

print('Starting')
matplotlib.use('agg')

checkpoints = checkpoint.parent.glob(checkpoint.name + '_*.pth')
checkpoints = [c for c in checkpoints if extract_id(c) in decoders]
assert len(checkpoints) >= 1, "No checkpoints found."

print(checkpoint.parent / 'args.pth')
#model_args = torch.load(checkpoint.parent / 'args.pth')[0]
encoder = wavenet_models.Encoder(model_args)
encoder.load_state_dict(torch.load(checkpoints[0])['encoder_state'])
encoder.eval()
encoder = encoder.cuda()

decoders = []
decoder_ids = []
for checkpoint in checkpoints:
    decoder = WaveNet(model_args)
    #decoder = nn.DataParallel(decoder)
    checkpoint_state = torch.load(checkpoint)['decoder_state']
    #pop_list = ["layers.40.causal.weight", "layers.40.causal.bias", "layers.40.condition.weight", "layers.40.condition.bias", "layers.40.residual.weight", "layers.40.residual.bias", "layers.40.skip.weight", "layers.40.skip.bias", "layers.41.causal.weight", "layers.41.causal.bias", "layers.41.condition.weight", "layers.41.condition.bias", "layers.41.residual.weight", "layers.41.residual.bias", "layers.41.skip.weight", "layers.41.skip.bias", "layers.42.causal.weight", "layers.42.causal.bias", "layers.42.condition.weight", "layers.42.condition.bias", "layers.42.residual.weight", "layers.42.residual.bias", "layers.42.skip.weight", "layers.42.skip.bias", "layers.43.causal.weight", "layers.43.causal.bias", "layers.43.condition.weight", "layers.43.condition.bias", "layers.43.residual.weight", "layers.43.residual.bias", "layers.43.skip.weight", "layers.43.skip.bias", "layers.44.causal.weight", "layers.44.causal.bias", "layers.44.condition.weight", "layers.44.condition.bias", "layers.44.residual.weight", "layers.44.residual.bias", "layers.44.skip.weight", "layers.44.skip.bias", "layers.45.causal.weight", "layers.45.causal.bias", "layers.45.condition.weight", "layers.45.condition.bias", "layers.45.residual.weight", "layers.45.residual.bias", "layers.45.skip.weight", "layers.45.skip.bias", "layers.46.causal.weight", "layers.46.causal.bias", "layers.46.condition.weight", "layers.46.condition.bias", "layers.46.residual.weight", "layers.46.residual.bias", "layers.46.skip.weight", "layers.46.skip.bias", "layers.47.causal.weight", "layers.47.causal.bias", "layers.47.condition.weight", "layers.47.condition.bias", "layers.47.residual.weight", "layers.47.residual.bias", "layers.47.skip.weight", "layers.47.skip.bias", "layers.48.causal.weight", "layers.48.causal.bias", "layers.48.condition.weight", "layers.48.condition.bias", "layers.48.residual.weight", "layers.48.residual.bias", "layers.48.skip.weight", "layers.48.skip.bias", "layers.49.causal.weight", "layers.49.causal.bias", "layers.49.condition.weight", "layers.49.condition.bias", "layers.49.residual.weight", "layers.49.residual.bias", "layers.49.skip.weight", "layers.49.skip.bias", "layers.50.causal.weight", "layers.50.causal.bias", "layers.50.condition.weight", "layers.50.condition.bias", "layers.50.residual.weight", "layers.50.residual.bias", "layers.50.skip.weight", "layers.50.skip.bias", "layers.51.causal.weight", "layers.51.causal.bias", "layers.51.condition.weight", "layers.51.condition.bias", "layers.51.residual.weight", "layers.51.residual.bias", "layers.51.skip.weight", "layers.51.skip.bias", "layers.52.causal.weight", "layers.52.causal.bias", "layers.52.condition.weight", "layers.52.condition.bias", "layers.52.residual.weight", "layers.52.residual.bias", "layers.52.skip.weight", "layers.52.skip.bias", "layers.53.causal.weight", "layers.53.causal.bias", "layers.53.condition.weight", "layers.53.condition.bias", "layers.53.residual.weight", "layers.53.residual.bias", "layers.53.skip.weight", "layers.53.skip.bias", "layers.54.causal.weight", "layers.54.causal.bias", "layers.54.condition.weight", "layers.54.condition.bias", "layers.54.residual.weight", "layers.54.residual.bias", "layers.54.skip.weight", "layers.54.skip.bias", "layers.55.causal.weight", "layers.55.causal.bias", "layers.55.condition.weight", "layers.55.condition.bias", "layers.55.residual.weight", "layers.55.residual.bias", "layers.55.skip.weight", "layers.55.skip.bias"]
    #for items in pop_list:
      #  checkpoint_state.pop(items)
    #checkpoint_state.pop(list(["layers.40.causal.weight", "layers.40.causal.bias", "layers.40.condition.weight", "layers.40.condition.bias", "layers.40.residual.weight", "layers.40.residual.bias", "layers.40.skip.weight", "layers.40.skip.bias", "layers.41.causal.weight", "layers.41.causal.bias", "layers.41.condition.weight", "layers.41.condition.bias", "layers.41.residual.weight", "layers.41.residual.bias", "layers.41.skip.weight", "layers.41.skip.bias", "layers.42.causal.weight", "layers.42.causal.bias", "layers.42.condition.weight", "layers.42.condition.bias", "layers.42.residual.weight", "layers.42.residual.bias", "layers.42.skip.weight", "layers.42.skip.bias", "layers.43.causal.weight", "layers.43.causal.bias", "layers.43.condition.weight", "layers.43.condition.bias", "layers.43.residual.weight", "layers.43.residual.bias", "layers.43.skip.weight", "layers.43.skip.bias", "layers.44.causal.weight", "layers.44.causal.bias", "layers.44.condition.weight", "layers.44.condition.bias", "layers.44.residual.weight", "layers.44.residual.bias", "layers.44.skip.weight", "layers.44.skip.bias", "layers.45.causal.weight", "layers.45.causal.bias", "layers.45.condition.weight", "layers.45.condition.bias", "layers.45.residual.weight", "layers.45.residual.bias", "layers.45.skip.weight", "layers.45.skip.bias", "layers.46.causal.weight", "layers.46.causal.bias", "layers.46.condition.weight", "layers.46.condition.bias", "layers.46.residual.weight", "layers.46.residual.bias", "layers.46.skip.weight", "layers.46.skip.bias", "layers.47.causal.weight", "layers.47.causal.bias", "layers.47.condition.weight", "layers.47.condition.bias", "layers.47.residual.weight", "layers.47.residual.bias", "layers.47.skip.weight", "layers.47.skip.bias", "layers.48.causal.weight", "layers.48.causal.bias", "layers.48.condition.weight", "layers.48.condition.bias", "layers.48.residual.weight", "layers.48.residual.bias", "layers.48.skip.weight", "layers.48.skip.bias", "layers.49.causal.weight", "layers.49.causal.bias", "layers.49.condition.weight", "layers.49.condition.bias", "layers.49.residual.weight", "layers.49.residual.bias", "layers.49.skip.weight", "layers.49.skip.bias", "layers.50.causal.weight", "layers.50.causal.bias", "layers.50.condition.weight", "layers.50.condition.bias", "layers.50.residual.weight", "layers.50.residual.bias", "layers.50.skip.weight", "layers.50.skip.bias", "layers.51.causal.weight", "layers.51.causal.bias", "layers.51.condition.weight", "layers.51.condition.bias", "layers.51.residual.weight", "layers.51.residual.bias", "layers.51.skip.weight", "layers.51.skip.bias", "layers.52.causal.weight", "layers.52.causal.bias", "layers.52.condition.weight", "layers.52.condition.bias", "layers.52.residual.weight", "layers.52.residual.bias", "layers.52.skip.weight", "layers.52.skip.bias", "layers.53.causal.weight", "layers.53.causal.bias", "layers.53.condition.weight", "layers.53.condition.bias", "layers.53.residual.weight", "layers.53.residual.bias", "layers.53.skip.weight", "layers.53.skip.bias", "layers.54.causal.weight", "layers.54.causal.bias", "layers.54.condition.weight", "layers.54.condition.bias", "layers.54.residual.weight", "layers.54.residual.bias", "layers.54.skip.weight", "layers.54.skip.bias", "layers.55.causal.weight", "layers.55.causal.bias", "layers.55.condition.weight", "layers.55.condition.bias", "layers.55.residual.weight", "layers.55.residual.bias", "layers.55.skip.weight", "layers.55.skip.bias"]))
    #decoder.load_state_dict(torch.load(checkpoint)['decoder_state'])
    decoder.load_state_dict(checkpoint_state)
    decoder.eval()
    decoder = decoder.cuda()
    decoder = WavenetGenerator(decoder, 1, wav_freq=16000)

    decoders += [decoder]
    decoder_ids += [extract_id(checkpoint)]

Starting
..\checkpoints\pretrained_musicnet\args.pth


In [41]:
xs = []
file_paths_gen = [Path('test.wav')]
for file_path in file_paths:
    data, rate = librosa.load(file_path, sr=16000)
    assert rate == 16000
    data = utils.mu_law(data)
    xs.append(torch.tensor(data).unsqueeze(0).float().cuda())

xs = torch.stack(xs).contiguous()
print(xs)
print(f'xs size: {xs.size()}')

tensor([[[100., 102., 101.,  ..., 129., 129., 131.]]], device='cuda:0')
xs size: torch.Size([1, 1, 80000])


In [42]:
xs = []

for file_path in file_paths:
    data, rate = librosa.load(file_path, sr=16000)
    assert rate == 16000
    data = utils.mu_law(data)
    print(data)
    xs.append(torch.tensor(data).unsqueeze(0).float().cuda())

xs = torch.stack(xs).contiguous()
print(f'xs size: {xs.size()}')

[100 102 101 ... 129 129 131]
xs size: torch.Size([1, 1, 80000])


In [ ]:
yy = {}
with torch.no_grad():
    zz = []
    for xs_batch in torch.split(xs, batch_size):
        zz += [encoder(xs_batch)]
    zz = torch.cat(zz, dim=0)

    with utils.timeit("Generation timer"):
        for i, decoder_id in enumerate(decoder_ids):
            yy[decoder_id] = []
            decoder = decoders[i]
            for zz_batch in torch.split(zz, batch_size):
                print(zz_batch.shape)
                splits = torch.split(zz_batch, split_size, -1)
                audio_data = []
                decoder.reset()
                for cond in tqdm.tqdm_notebook(splits):
                    audio_data += [decoder.generate(cond).cuda()]
                    print(decoder.generate(cond).cuda().shape)
                audio_data = torch.cat(audio_data, -1)
                yy[decoder_id] += [audio_data]
                
            yy[decoder_id] = torch.cat(yy[decoder_id], dim=0)

In [35]:
for decoder_ix, decoder_result in yy.items():
    for sample_result, filepath in zip(decoder_result, file_paths):
        disp(sample_result, decoder_ix)